In [1]:
import pandas as pd
import os
import glob
import shutil
import random
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
import numpy as np

In [2]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented'

work_dir = '/Users/amandeep/Github/table-linker/data/t2dv2'
# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-canonical/
train_path = f'{work_dir}/t2dv2-train-canonical/'
# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-canonical/
dev_path = f'{work_dir}/t2dv2-dev-canonical/'

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-candidates-dwd-v2/
train_candidate_path = f'{train_path}output-candidates/'
train_feature_path = f'{train_path}output-features-2/'
# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-candidates-dwd-v2/
dev_candidate_path = f'{dev_path}output-candidates/'
dev_feature_path = f'{dev_path}output-features-2/'

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/ground_truth/Xinting_GT_csv
ground_truth_files = f'{work_dir}/round_1_GT/'
classifier_model_path = '/Users/amandeep/Github/table-linker-pipelines/table-linker-full-pipeline/models/weighted_lr.pkl'

In [3]:
aux_field = 'graph_embedding_complex,class_count,property_count'
temp_dir = f'{work_dir}/temp/' #temp directory to store intermediate files

#directory to store the property count file for each table. Can be directly used for computing the tf-idf features 
#without running the candidate generation process again which is expensive

#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/train_prop_count/
train_prop_count = f'{temp_dir}/train_prop_count/' 
#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/dev_prop_count/
dev_prop_count = f'{temp_dir}/dev_prop_count/'

#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/train_class_count/
train_class_count = f'{temp_dir}/train_class_count/'
#GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/dev_class_count/
dev_class_count = f'{temp_dir}/dev_class_count/'

train_graph_embedding = f'{temp_dir}/train_graph_embedding/'
dev_graph_embedding = f'{temp_dir}/dev_graph_embedding/'




In [4]:
!mkdir -p $temp_dir
!mkdir -p $train_prop_count
!mkdir -p $dev_prop_count
!mkdir -p $train_class_count
!mkdir -p $dev_class_count
!mkdir -p $train_graph_embedding
!mkdir -p $dev_graph_embedding
!mkdir -p $train_candidate_path
!mkdir -p $dev_candidate_path
!mkdir -p $train_feature_path
!mkdir -p $dev_feature_path

In [4]:
def candidate_generation(path, gt_path, output_path, class_count, prop_count, graph_embedding):
    for i, file in enumerate(glob.glob(path + '*.csv')):
        st = time.time()
        filename = file.split('/')[-1]
        print(filename)
        gt_file = os.path.join(ground_truth_files, filename)
        output_file = os.path.join(output_path, filename)
        
        !tl clean -c label -o label_clean $file / \
        --url $es_url --index $es_index \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder $temp_dir / \
        --url $es_url --index $es_index \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder {temp_dir} / \
        ground-truth-labeler --gt-file $gt_file > $output_file
        
        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = os.path.join(class_count, filename.strip('.csv') + '_class_count.tsv')
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = os.path.join(prop_count, filename.strip('.csv') + '_prop_count.tsv')
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            else:
                graph_embedding_file = os.path.join(graph_embedding, filename.strip('.csv') + '_graph_embedding_complex.tsv')
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)
        
        print(time.time() - st)



In [5]:
def feature_generation(candidate_dir, embedding_dir, class_count_dir, property_count_dir, output_path):
    print(candidate_dir)
    print(embedding_dir)
    print(class_count_dir)
    print(property_count_dir)
    print(output_path)
    for file in glob.glob(candidate_dir + '*.csv'):
        filename = file.split('/')[-1]
        print(filename)
        embedding_file = os.path.join(embedding_dir, filename.strip('.csv') + '_graph_embedding_complex.tsv')
        class_count_file = f"{class_count_dir}{filename.strip('.csv')}_class_count.tsv"
        property_count_file = f"{property_count_dir}{filename.strip('.csv')}_prop_count.tsv"
        output_file = os.path.join(output_path, filename)
        !tl align-page-rank $file \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases \
            / string-similarity -i --method jaro_winkler -o jaro_winkler \
            / string-similarity -i --method levenshtein -o levenshtein \
            / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
            / normalize-scores -c des_cont_jaccard / smallest-qnode-number \
            / mosaic-features -c kg_labels --num-char --num-tokens \
            / create-singleton-feature -o singleton \
            / vote-by-classifier  \
            --prob-threshold 0.995 \
            --model $classifier_model_path \
            / score-using-embedding \
            --column-vector-strategy centroid-of-lof \
            --lof-strategy ems-mv \
            -o lof-graph-embedding-score \
            --embedding-file $embedding_file \
            --embedding-url "$es_url/$es_index/" \
            / generate-reciprocal-rank  \
            -c lof-graph-embedding-score \
            -o lof-reciprocal-rank \
            / compute-tf-idf  \
            --feature-file $class_count_file \
            --feature-name class_count \
            --singleton-column singleton \
            -o lof_class_count_tf_idf_score \
            / compute-tf-idf \
            --feature-file $property_count_file \
            --feature-name property_count \
            --singleton-column singleton \
            -o lof_property_count_tf_idf_score \
            > $output_file

In [ ]:
['pagerank','retrieval_score','monge_elkan','des_cont_jaccard',
            'jaro_winkler','levenshtein','singleton','is_lof','num_char','num_tokens',
           'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
           'lof-graph-embedding-score', 'lof-reciprocal-rank']

In [13]:
candidate_generation(train_path, ground_truth_files, train_candidate_path, train_class_count, train_prop_count, train_graph_embedding)

58891288_0_1117541047012405958.csv
215.0770800113678
39173938_0_7916056990138658530.csv
204.2746980190277
10579449_0_1681126353774891032.csv
41.339812994003296
33401079_0_9127583903019856402.csv
195.1763949394226
21362676_0_6854186738074119688.csv
225.82092714309692
38428277_0_1311643810102462607.csv
247.50702118873596
91959037_0_7907661684242014480.csv
865.6964118480682
20135078_0_7570343137119682530.csv
215.20253586769104
35188621_0_6058553107571275232.csv
230.3643729686737
54719588_0_8417197176086756912.csv
553.5450069904327
21245481_0_8730460088443117515.csv
480.3478729724884
71840765_0_6664391841933033844.csv
35.67017602920532
8468806_0_4382447409703007384.csv
167.83368825912476
88523363_0_8180214313099580515.csv
1107.9533722400665
29414811_13_8724394428539174350.csv
38.97614812850952
99070098_0_2074872741302696997.csv
449.4610207080841
43237185_1_3636357855502246981.csv
67.81162977218628
46671561_0_6122315295162029872.csv
484.14470076560974
53989675_0_8697482470743954630.csv
89.1

In [14]:
candidate_generation(dev_path, ground_truth_files, dev_candidate_path, dev_class_count, dev_prop_count, dev_graph_embedding)

39759273_0_1427898308030295194.csv
221.68012309074402
45073662_0_3179937335063201739.csv
71.87427115440369
29414811_2_4773219892816395776.csv
50.534576177597046
84575189_0_6365692015941409487.csv
272.8969089984894
14380604_4_3329235705746762392.csv
46.958092212677
52299421_0_4473286348258170200.csv
195.72647094726562
50270082_0_444360818941411589.csv
379.41621804237366
28086084_0_3127660530989916727.csv
425.98705887794495
14067031_0_559833072073397908.csv
109.25832009315491


In [6]:
feature_generation(train_candidate_path, train_graph_embedding, train_class_count, train_prop_count, train_feature_path)

/Users/amandeep/Github/table-linker/data/t2dv2/t2dv2-train-canonical/output-candidates/
/Users/amandeep/Github/table-linker/data/t2dv2/temp//train_graph_embedding/
/Users/amandeep/Github/table-linker/data/t2dv2/temp//train_class_count/
/Users/amandeep/Github/table-linker/data/t2dv2/temp//train_prop_count/
/Users/amandeep/Github/table-linker/data/t2dv2/t2dv2-train-canonical/output-features-2/
58891288_0_1117541047012405958.csv
Qnodes to lookup: 10717
Qnodes from file: 10399
Qnodes from server: 0
Outlier removal generates 86 lof-voted candidates
39173938_0_7916056990138658530.csv
Qnodes to lookup: 9986
Qnodes from file: 9718
Qnodes from server: 0
Outlier removal generates 73 lof-voted candidates
10579449_0_1681126353774891032.csv
Qnodes to lookup: 1706
Qnodes from file: 1652
Qnodes from server: 0
Command: score-using-embedding
Error Message:  Traceback (most recent call last):
  File "/Users/amandeep/Github/table-linker/tl/cli/score-using-embedding.py", line 74, in run
    vector_transfo

In [7]:
feature_generation(dev_candidate_path, dev_graph_embedding, dev_class_count, dev_prop_count, dev_feature_path)

/Users/amandeep/Github/table-linker/data/t2dv2/t2dv2-dev-canonical/output-candidates/
/Users/amandeep/Github/table-linker/data/t2dv2/temp//dev_graph_embedding/
/Users/amandeep/Github/table-linker/data/t2dv2/temp//dev_class_count/
/Users/amandeep/Github/table-linker/data/t2dv2/temp//dev_prop_count/
/Users/amandeep/Github/table-linker/data/t2dv2/t2dv2-dev-canonical/output-features-2/
39759273_0_1427898308030295194.csv
Qnodes to lookup: 10448
Qnodes from file: 10120
Qnodes from server: 0
Outlier removal generates 101 lof-voted candidates
45073662_0_3179937335063201739.csv
Qnodes to lookup: 3040
Qnodes from file: 3004
Qnodes from server: 0
Command: score-using-embedding
Error Message:  Traceback (most recent call last):
  File "/Users/amandeep/Github/table-linker/tl/cli/score-using-embedding.py", line 74, in run
    vector_transformer.process_vectors()
  File "/Users/amandeep/Github/table-linker/tl/features/external_embedding.py", line 158, in process_vectors
    if not self._centroid_of_l

### Generate Balanced Training Data

In [49]:
training_datapath = '../random_forest_ranking/training_data_dwd.csv'

In [43]:
final_list = []
for i,file in enumerate(glob.glob(train_candidate_path + '*.csv')):
    file_name = file.split('/')[-1]
    print(file_name)
    
    try:
        d_sample = pd.read_csv(file)
        grouped_obj = d_sample.groupby(['row', 'column'])
        for cell in grouped_obj:
            num_rows = random.randint(2,5)
            sorted_df = cell[1].sort_values('graph-embedding-score',ascending=False)
            if 0 in sorted_df['evaluation_label'].tolist():
                continue
            if sorted_df.empty:
                continue
            if num_rows < len(sorted_df):
                top_sample_df = sorted_df[sorted_df['evaluation_label'] == -1][:10].sample(n=num_rows)
                bottom_sample_df = sorted_df[sorted_df['evaluation_label'] == -1][-10:].sample(n=num_rows)
                final_list.extend(top_sample_df.to_dict(orient='records'))
                final_list.extend(bottom_sample_df.to_dict(orient='records'))
            else:
                sample_df = sorted_df[sorted_df['evaluation_label'] == -1]
                final_list.extend(sample_df.to_dict(orient='records'))
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
            final_list.extend(a.to_dict(orient='records'))
    except: 
        pass

train_df = pd.DataFrame(final_list)

58891288_0_1117541047012405958.csv
39173938_0_7916056990138658530.csv
10579449_0_1681126353774891032.csv
33401079_0_9127583903019856402.csv
21362676_0_6854186738074119688.csv
38428277_0_1311643810102462607.csv
91959037_0_7907661684242014480.csv
20135078_0_7570343137119682530.csv
35188621_0_6058553107571275232.csv
54719588_0_8417197176086756912.csv
21245481_0_8730460088443117515.csv
71840765_0_6664391841933033844.csv
8468806_0_4382447409703007384.csv
88523363_0_8180214313099580515.csv
29414811_13_8724394428539174350.csv
99070098_0_2074872741302696997.csv
43237185_1_3636357855502246981.csv
46671561_0_6122315295162029872.csv
53989675_0_8697482470743954630.csv
25404227_0_2240631045609013057.csv
9834884_0_3871985887467090123.csv
63450419_0_8012592961815711786.csv
1438042986423_95_20150728002306-00125-ip-10-236-191-2_88435628_5.csv
22864497_0_8632623712684511496.csv
53822652_0_5767892317858575530.csv
37856682_0_6818907050314633217.csv
26310680_0_5150772059999313798.csv
29414811_12_2511524702

In [50]:
train_df.to_csv(training_datapath, index=False)

### Data Exploration

In [4]:
df = pd.read_csv(train_datapath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48365 entries, 0 to 48364
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   column                 48365 non-null  int64  
 1   row                    48365 non-null  int64  
 2   label                  48363 non-null  object 
 3   context                48353 non-null  object 
 4   label_clean            48363 non-null  object 
 5   kg_id                  47784 non-null  object 
 6   kg_labels              46828 non-null  object 
 7   kg_aliases             14414 non-null  object 
 8   method                 48365 non-null  object 
 9   kg_descriptions        39166 non-null  object 
 10  pagerank               48365 non-null  float64
 11  retrieval_score        48365 non-null  float64
 12  GT_kg_id               48365 non-null  object 
 13  GT_kg_label            48365 non-null  object 
 14  evaluation_label       48365 non-null  int64  
 15  mo

In [5]:
# Features we need to include in training
features = ['pagerank','retrieval_score','monge_elkan',
            'des_cont_jaccard','jaro_winkler','graph-embedding-score',
           'singleton','num_char','num_tokens','reciprocal_rank']
evaluation_label = ['evaluation_label']

df[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48365 entries, 0 to 48364
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pagerank               48365 non-null  float64
 1   retrieval_score        48365 non-null  float64
 2   monge_elkan            48365 non-null  float64
 3   des_cont_jaccard       48365 non-null  float64
 4   jaro_winkler           48365 non-null  float64
 5   graph-embedding-score  48365 non-null  float64
 6   singleton              48365 non-null  int64  
 7   num_char               48365 non-null  int64  
 8   num_tokens             48365 non-null  int64  
 9   reciprocal_rank        48365 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 3.7 MB


In [6]:
df['graph-embedding-score'] = df['graph-embedding-score'].fillna(0.0)
df['reciprocal_rank'] = df['reciprocal_rank'].fillna(0.0)
df[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48365 entries, 0 to 48364
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pagerank               48365 non-null  float64
 1   retrieval_score        48365 non-null  float64
 2   monge_elkan            48365 non-null  float64
 3   des_cont_jaccard       48365 non-null  float64
 4   jaro_winkler           48365 non-null  float64
 5   graph-embedding-score  48365 non-null  float64
 6   singleton              48365 non-null  int64  
 7   num_char               48365 non-null  int64  
 8   num_tokens             48365 non-null  int64  
 9   reciprocal_rank        48365 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 3.7 MB


### Train a Random Forest Regressor

In [58]:
train_data = df[features]
y_label = df[evaluation_label]

In [61]:
model = RandomForestRegressor(n_estimators=100, max_features="log2",min_samples_leaf=3)
model.fit(train_data,y_label)
y_pred = model.predict(train_data)

/Users/rijulvohra/opt/anaconda3/envs/table_linker_dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [62]:
mean_squared_error(y_label, y_pred)

0.036428866339495325

In [7]:
model_save_path = '../random_forest_ranking/rf_tuned_dwd_ranking.pkl'


In [ ]:
pickle.dump(model,open(model_save_path,'wb'))

In [8]:
saved_model = pickle.load(open(model_save_path, 'rb'))

### Predicting Scores for Train set

In [9]:
train_pred_output = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-train-rf-pred-dwd-2/'

In [75]:
train_mse = []
for file in glob.glob(train_candidate_path + '*.csv'):
    try:
        file_name = file.split('/')[-1]
        print(file_name)
        df_file = pd.read_csv(file)
        data = df_file[features]
        y_file_label = df_file[evaluation_label]
        y_file_pred = saved_model.predict(data)
        df_file['rf_model_pred'] = y_file_pred
        file_mse = mean_squared_error(y_file_label,y_file_pred)
        train_mse.append(file_mse)
        df_file.to_csv(os.path.join(train_pred_output,file_name),index=False)
    except:
        pass

58891288_0_1117541047012405958.csv
39173938_0_7916056990138658530.csv
10579449_0_1681126353774891032.csv
33401079_0_9127583903019856402.csv
21362676_0_6854186738074119688.csv
38428277_0_1311643810102462607.csv
91959037_0_7907661684242014480.csv
20135078_0_7570343137119682530.csv
35188621_0_6058553107571275232.csv
54719588_0_8417197176086756912.csv
21245481_0_8730460088443117515.csv
71840765_0_6664391841933033844.csv
8468806_0_4382447409703007384.csv
88523363_0_8180214313099580515.csv
29414811_13_8724394428539174350.csv
99070098_0_2074872741302696997.csv
43237185_1_3636357855502246981.csv
46671561_0_6122315295162029872.csv
53989675_0_8697482470743954630.csv
25404227_0_2240631045609013057.csv
9834884_0_3871985887467090123.csv
63450419_0_8012592961815711786.csv
1438042986423_95_20150728002306-00125-ip-10-236-191-2_88435628_5.csv
22864497_0_8632623712684511496.csv
53822652_0_5767892317858575530.csv
37856682_0_6818907050314633217.csv
26310680_0_5150772059999313798.csv
29414811_12_2511524702

In [76]:
print("Train MSE is: ", sum(train_mse)/len(train_mse))

Train MSE is:  0.4519201624197591


### Predicting Scores for dev set

In [10]:
dev_pred_output = '/Users/rijulvohra/Documents/work/Novartis-ISI/novartis-isi-git/entity_linking/t2dv2-raw/t2dv2/canonical-with-context/t2dv2-dev-rf-pred-dwd-2/'

In [19]:
dev_mse = []
for file in glob.glob(dev_candidate_path + '*.csv'):

    file_name = file.split('/')[-1]
    print(file_name)
    df_file = pd.read_csv(file)
    data = df_file[features]
    y_file_label = df_file[evaluation_label]
    y_file_pred = saved_model.predict(data)
    df_file['rf_model_pred'] = y_file_pred
    file_mse = mean_squared_error(y_file_label,y_file_pred)
    dev_mse.append(file_mse)
    df_file.to_csv(os.path.join(dev_pred_output,file_name),index=False)
    

39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


In [20]:
print("Dev MSE is: ", sum(dev_mse)/len(dev_mse))

Dev MSE is:  0.40129949369858403


### Evaluation

In [10]:
final_score_path = train_pred_output

In [11]:
import os
eval_file_names = []
for (dirpath, dirnames, filenames) in os.walk(final_score_path):
    for fn in filenames:
        if "csv" not in fn:
            continue
        abs_fn = dirpath + fn
        assert os.path.isfile(abs_fn)
        if os.path.getsize(abs_fn) == 0:
            continue
        eval_file_names.append(abs_fn)
len(eval_file_names)

44

In [12]:
# merge all eval files in one df
def merge_df(file_names: list):
    df_list = []
    for fn in file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        # df = df.fillna('')
        df_list.append(df)
    return pd.concat(df_list)
all_data = merge_df(eval_file_names)
all_data

,column,row,label,context,label_clean,kg_id,kg_labels,kg_aliases,method,kg_descriptions,...,monge_elkan,des_cont_jaccard,jaro_winkler,graph-embedding-score,singleton,reciprocal_rank,num_char,num_tokens,rf_model_pred,table_id
0,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q28452137,Godfather,NaN,fuzzy-augmented,album by Wiley,...,0.750000,0.000000,0.897436,0.350448,0,0.009709,9,1,-0.976667,58891288_0_1117541047012405958
1,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q1066512,Charles Wright,The Godfather|Papa Shango,fuzzy-augmented,American professional wrestler,...,0.499339,0.000000,0.584249,0.290255,0,0.008929,14,2,-1.000000,58891288_0_1117541047012405958
2,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q47703,The Godfather,Godfather|The Godfather Part I,fuzzy-augmented,1972 American film directed by Francis Ford Co...,...,1.000000,0.375000,1.000000,0.783063,0,1.000000,13,2,0.996296,58891288_0_1117541047012405958
3,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q6144534,The Godfather,NaN,fuzzy-augmented,2002 album,...,1.000000,0.000000,1.000000,0.468657,0,0.011905,13,2,0.930031,58891288_0_1117541047012405958
4,1,0,The Godfather,1|1972|Francis Ford Coppola|1,The Godfather,Q5205330,DJ Godfather,NaN,fuzzy-augmented,American DJ and record producer,...,0.500000,0.000000,0.800855,0.257788,0,0.008547,12,2,-1.000000,58891288_0_1117541047012405958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33011,0,99,Groundhog Day,1993|DVD|Harold Ramis|Danny Rubin|101|Widescreen,Groundhog Day,Q1210286,Groundhog Day,NaN,exact-match,Wikimedia disambiguation page,...,1.000000,0.000000,1.000000,0.479545,0,0.018182,13,2,0.676114,50245608_0_871275842592178099
33012,0,99,Groundhog Day,1993|DVD|Harold Ramis|Danny Rubin|101|Widescreen,Groundhog Day,Q18614990,Groundhog Day,xkcd 1076,exact-match,1076th strip of the webcomic xkcd,...,1.000000,0.000000,1.000000,0.611415,0,0.071429,13,2,0.300482,50245608_0_871275842592178099
33013,0,99,Groundhog Day,1993|DVD|Harold Ramis|Danny Rubin|101|Widescreen,Groundhog Day,Q19961225,Groundhog Day,NaN,exact-match,Musical comedy based on the film of the same name,...,1.000000,0.000000,1.000000,0.644078,0,0.111111,13,2,-0.976559,50245608_0_871275842592178099
33014,0,99,Groundhog Day,1993|DVD|Harold Ramis|Danny Rubin|101|Widescreen,Groundhog Day,Q488655,Groundhog Day,\\ Groundhog Day \\,exact-match,1993 comedy film directed by Harold Ramis,...,1.000000,0.285714,1.000000,0.815621,0,0.500000,13,2,1.000000,50245608_0_871275842592178099


In [13]:
# parse eval file
from pandas.core.common import SettingWithCopyError
pd.options.mode.chained_assignment = 'raise'

def parse_eval_files_stats(eval_data,method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'row', 'column'])['table_id'].count().reset_index(name="count")
    res['num_tasks'] = len(eval_data.groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_gt_in_candidate'] = len(eval_data[eval_data['evaluation_label'] == 1].groupby(['table_id', 'row', 'column']))
    res['num_tasks_with_singleton_candidate'] = len(candidate_eval_data[candidate_eval_data['count'] == 1].groupby(['table_id', 'row', 'column']))
    singleton_eval_data = candidate_eval_data[candidate_eval_data['count'] == 1]
    num_tasks_with_singleton_candidate_with_gt = 0
    for i, row in singleton_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) == 1
        if c_e_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_singleton_candidate_with_gt += 1
    res['num_tasks_with_singleton_candidate_with_gt'] = num_tasks_with_singleton_candidate_with_gt
    num_tasks_with_graph_top_one_accurate = []
    num_tasks_with_graph_top_five_accurate = []
    num_tasks_with_graph_top_ten_accurate = []
    num_tasks_with_final_score_top_one_accurate = []
    num_tasks_with_final_score_top_five_accurate = []
    num_tasks_with_final_score_top_ten_accurate = []
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    ndcg_score_g_list = []
    ndcg_model_score_list = []
    has_gt_list = []
    has_gt_in_candidate = []
    # candidate_eval_data = candidate_eval_data[:1]
    for i, row in candidate_eval_data.iterrows():
        #print(i)
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
            
        # handle graph-embedding-score
        s_data = c_e_data.sort_values(by=['graph-embedding-score'], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_graph_top_one_accurate.append(1)
        else:
            num_tasks_with_graph_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_graph_top_five_accurate.append(1)
        else:
            num_tasks_with_graph_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_graph_top_ten_accurate.append(1)
        else:
            num_tasks_with_graph_top_ten_accurate.append(0)
        
        #rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
        cf_e_data = c_e_data.copy()
        #cf_e_data['evaluation_label'] = cf_e_data['evaluation_label'].replace(-1, 0)
#         cf_e_data['text-embedding-score'] = cf_e_data['text-embedding-score'].replace(np.nan, 0)
        cf_e_data['graph-embedding-score'] = cf_e_data['graph-embedding-score'].replace(np.nan, 0)
        cf_e_data[method] = cf_e_data[method].replace(np.nan, 0)

    candidate_eval_data['graph_top_one_accurate'] = num_tasks_with_graph_top_one_accurate
    candidate_eval_data['graph_top_five_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['graph_top_ten_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['model_top_one_accurate'] = num_tasks_with_model_score_top_one_accurate
    candidate_eval_data['model_top_five_accurate'] = num_tasks_with_model_score_top_five_accurate
    candidate_eval_data['model_top_ten_accurate'] = num_tasks_with_model_score_top_ten_accurate
    candidate_eval_data['has_gt'] = has_gt_list
    candidate_eval_data['has_gt_in_candidate'] = has_gt_in_candidate
    res['num_tasks_with_graph_top_one_accurate'] = sum(num_tasks_with_graph_top_one_accurate)
    res['num_tasks_with_graph_top_five_accurate'] = sum(num_tasks_with_graph_top_five_accurate)
    res['num_tasks_with_graph_top_ten_accurate'] = sum(num_tasks_with_graph_top_ten_accurate)
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data

In [14]:
res, candidate_eval_data = parse_eval_files_stats(all_data,'rf_model_pred')
print(res)
display(candidate_eval_data)

{'num_tasks': 6684, 'num_tasks_with_gt': 5822, 'num_tasks_with_gt_in_candidate': 5620, 'num_tasks_with_singleton_candidate': 0, 'num_tasks_with_singleton_candidate_with_gt': 0, 'num_tasks_with_graph_top_one_accurate': 1954, 'num_tasks_with_graph_top_five_accurate': 3438, 'num_tasks_with_graph_top_ten_accurate': 3978, 'num_tasks_with_model_score_top_one_accurate': 4574, 'num_tasks_with_model_score_top_five_accurate': 5184, 'num_tasks_with_model_score_top_ten_accurate': 5312}


,table_id,row,column,count,graph_top_one_accurate,graph_top_five_accurate,graph_top_ten_accurate,model_top_one_accurate,model_top_five_accurate,model_top_ten_accurate,has_gt,has_gt_in_candidate
0,10579449_0_1681126353774891032,0,1,101,0,1,1,1,1,1,1,1
1,10579449_0_1681126353774891032,1,1,101,0,0,0,1,1,1,1,1
2,10579449_0_1681126353774891032,2,1,101,0,0,0,0,0,0,0,0
3,10579449_0_1681126353774891032,3,1,100,0,0,0,0,0,0,0,0
4,10579449_0_1681126353774891032,4,1,50,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6679,99070098_0_2074872741302696997,209,1,101,0,1,1,1,1,1,1,1
6680,99070098_0_2074872741302696997,210,1,101,1,1,1,1,1,1,1,1
6681,99070098_0_2074872741302696997,211,1,124,0,1,1,0,1,1,1,1
6682,99070098_0_2074872741302696997,212,1,107,1,1,1,1,1,1,1,1


In [15]:
# Conclusion of exact-match on all tasks with ground truth (no filtering)
print(f"number of tasks: {res['num_tasks']}")
print(f"number of tasks with ground truth: {res['num_tasks_with_gt']}")
print(f"number of tasks with ground truth in candidate set: {res['num_tasks_with_gt_in_candidate']}, which is {res['num_tasks_with_gt_in_candidate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks has singleton candidate set: {res['num_tasks_with_singleton_candidate']}, which is {res['num_tasks_with_singleton_candidate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks has singleton candidate set which is ground truth: {res['num_tasks_with_singleton_candidate_with_gt']}, which is {res['num_tasks_with_singleton_candidate_with_gt']/res['num_tasks_with_gt'] * 100}%")
print()
print(f"number of tasks with top-1 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_one_accurate']}, which is {res['num_tasks_with_graph_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-5 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_five_accurate']}, which is {res['num_tasks_with_graph_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-10 accuracy in terms of graph embedding score: {res['num_tasks_with_graph_top_ten_accurate']}, which is {res['num_tasks_with_graph_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
print()
print(f"number of tasks with top-1 accuracy in terms of model score: {res['num_tasks_with_model_score_top_one_accurate']}, which is {res['num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-5 accuracy in terms of model score: {res['num_tasks_with_model_score_top_five_accurate']}, which is {res['num_tasks_with_model_score_top_five_accurate']/res['num_tasks_with_gt'] * 100}%")
print(f"number of tasks with top-10 accuracy in terms of model score: {res['num_tasks_with_model_score_top_ten_accurate']}, which is {res['num_tasks_with_model_score_top_ten_accurate']/res['num_tasks_with_gt'] * 100}%")
print()
candidate_eval_data_with_gt = candidate_eval_data[candidate_eval_data['has_gt'] == 1]

number of tasks: 6684
number of tasks with ground truth: 5822
number of tasks with ground truth in candidate set: 5620, which is 96.53040192373756%
number of tasks has singleton candidate set: 0, which is 0.0%
number of tasks has singleton candidate set which is ground truth: 0, which is 0.0%

number of tasks with top-1 accuracy in terms of graph embedding score: 1954, which is 33.56234970800413%
number of tasks with top-5 accuracy in terms of graph embedding score: 3438, which is 59.05187220886293%
number of tasks with top-10 accuracy in terms of graph embedding score: 3978, which is 68.3270353830299%

number of tasks with top-1 accuracy in terms of model score: 4574, which is 78.56406733081415%
number of tasks with top-5 accuracy in terms of model score: 5184, which is 89.04156647200276%
number of tasks with top-10 accuracy in terms of model score: 5312, which is 91.24012366884232%



In [27]:
c = candidate_eval_data.groupby(['table_id']).agg({
    'graph_top_one_accurate':lambda x: sum(x)/len(x),
    'model_top_one_accurate':lambda x: sum(x)/len(x),
    'graph_top_five_accurate':lambda x: sum(x)/len(x),
    'model_top_five_accurate':lambda x: sum(x)/len(x)
})
c['table type'] = [
    'country II',
    'companies',
    'pope',
    'video games',
    'movies',
    'players I',
    'players II',
    'magazines'
]
c

,graph_top_one_accurate,model_top_one_accurate,graph_top_five_accurate,model_top_five_accurate,table type
table_id,,,,,
14067031_0_559833072073397908,0.886792,0.943396,0.943396,0.943396,country II
14380604_4_3329235705746762392,0.700000,0.700000,0.900000,0.800000,companies
28086084_0_3127660530989916727,0.236607,0.611607,0.522321,0.723214,pope
29414811_2_4773219892816395776,0.136364,0.545455,0.500000,0.863636,video games
39759273_0_1427898308030295194,0.580000,0.880000,0.930000,0.970000,movies
45073662_0_3179937335063201739,0.407407,0.629630,0.888889,0.740741,players I
50270082_0_444360818941411589,0.547619,0.744048,0.922619,0.910714,players II
84575189_0_6365692015941409487,0.110000,0.700000,0.230000,0.780000,magazines
